In [7]:
import collections
import pprint
import re

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [8]:
urls = {"0": "https://www.gutenberg.org/cache/epub/25326/pg25326-images.html",
        "1": "https://www.gutenberg.org/cache/epub/25759/pg25759-images.html",
        "2": "https://www.gutenberg.org/cache/epub/26860/pg26860-images.html",
        "3": "https://www.gutenberg.org/cache/epub/28420/pg28420-images.html",
        "4": "https://www.gutenberg.org/cache/epub/28421/pg28421-images.html",
        "5": "https://www.gutenberg.org/cache/epub/28422/pg28422-images.html",
        "6": "https://www.gutenberg.org/cache/epub/31845/pg31845-images.html",
        "7": "https://www.gutenberg.org/cache/epub/31938/pg31938-images.html",
        "8": "https://www.gutenberg.org/cache/epub/32362/pg32362-images.html",
        "9": "https://www.gutenberg.org/cache/epub/33203/pg33203-images.html"}

In [9]:
page_number_pattern = re.compile(r'(\[Pg (\d+)\])')
for url in urls.keys():
    print(f"Working on {url}.")
    response = requests.get(urls[url])
    html_content = response.content

    soup = BeautifulSoup(html_content, "html.parser")

    pages_dict = {}
    page_data = {}
    current_page = None
    text_buffer = ""
    text_buffer_before = ""
    previous_page = None
    paragraph_index = 1

    for p in soup.find_all(['p', 'a', 'span']):
        if p.name == 'p':
            if not p.find_parents("div", class_="figcenter") and not p.find_parents("div", class_="footnote"):
                text = p.get_text()
            else:
                continue
        elif p.name == 'a' or p.name == 'span':
            if p.parent.get('class') == ['pagenum'] or p.get('class') == ['pagenum']:
                text = p.get_text()
            else:
                continue
        text = re.sub(r"(\r)?\n", " ", text)
        matches = page_number_pattern.findall(text)
        #print(f"We're at {current_page}, with current paragraph {paragraph_index} and matches {matches} and text {text}.")
        if len(matches) == 0 and current_page:
            page_data[current_page]['paragraph_ids'].append(paragraph_index)
            page_data[current_page]['text'].append(text)
            if text != '':
                paragraph_index += 1
        elif len(list(matches)) >= 1:
            if re.fullmatch(r'(\[Pg \d+\])+', text):
                for match in matches:
                    current_page = match[int(1)]

                    if not current_page in page_data.keys():
                        page_data[current_page] = {'paragraph_ids': [''], 'text': ['']}

            else:
                previous_page = current_page
                segments = re.split(fr"\[Pg [a-z0-9]+\]", text)
                i = 1
                try:
                    page_data[previous_page]['paragraph_ids'].append(paragraph_index)
                    page_data[previous_page]['text'].append(segments[0])
                except KeyError:
                    print(f"Current page is {current_page} and previous page is {previous_page}.")
                    
                for match in matches:
                    current_page = match[int(1)]

                    if not current_page in page_data.keys():
                        page_data[current_page] = {'paragraph_ids': [paragraph_index], 'text': [segments[i]]}
                    else:
                        page_data[current_page]['paragraph_ids'].append(paragraph_index)
                        page_data[current_page]['text'].append(segments[i])

                    if i < len(list(matches)):
                        i += 1

                paragraph_index += 1

    page_data = {(int(k)): v for (k, v) in page_data.items()}
    df = pd.DataFrame.from_dict(page_data).transpose().reset_index().rename(columns={'index': 'page'})
    df2 = df.explode(['paragraph_ids', 'text'])
    df2 = df2.rename(columns={'paragraph_ids': 'paragraph_id'})
    
    df2.to_csv(f"../../data/volumes/{url}.csv", index=False)

Working on 0.
Working on 1.
Working on 2.
Working on 3.
Working on 4.
Working on 5.
Working on 6.
Working on 7.
Working on 8.
Working on 9.
